# Data Wrangling

This notebook is dedicated to wrangling data from the subreddits `r/lifehacks` and `r/LifeProTips` using the Pushshift Reddit API.

## Problem Statements

> 1. Develop a classification model that can predict whether a reddit post belongs to the subreddits r/LifeProTips or r/Lifehacks, based on the content of the post

> 2. Identify the top 10 keywords that distinguish r/LifeProTips and r/Lifehacks

These two problem statements will guide the analysis of this study.

## Library Imports

In [39]:
%run 00_Workflow_Functions.ipynb import na_only, api_call, data_wrangling

In [40]:
import pandas as pd
import requests
from collections import defaultdict

## API Call Tests

In [41]:
# lifehacks endpoint
lhs_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=lifehacks&metadata=true&size=0"
lht_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=lifehacks&metadata=true&size=0&is_self=true"

# lifeprotips endpoint
lpts_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&metadata=true&size=0"
lptt_url = "https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&metadata=true&size=0&is_self=true"

In [42]:
# verifying call was successful
res_lhs = requests.get(lhs_url)
res_lht = requests.get(lht_url)
res_lpts = requests.get(lpts_url)
res_lptt = requests.get(lptt_url)

print(res_lhs.status_code, res_lht.status_code)
print(res_lpts.status_code, res_lptt.status_code)

200 200
200 200


All requests successful.

In [43]:
lhs_count = res_lhs.json()['metadata']['total_results']
lht_count = res_lht.json()['metadata']['total_results']

lpts_count = res_lpts.json()['metadata']['total_results']
lptt_count = res_lptt.json()['metadata']['total_results']

print(f"LifeHacks Total Submissions: {lhs_count}\nLifeHacks Total Self-Text Posts: {lht_count}")
print(f"LifeProTips Total Submissions: {lpts_count}\nLifeProTips Total Self-Text Posts: {lptt_count}")

LifeHacks Total Submissions: 81966
LifeHacks Total Self-Text Posts: 23329
LifeProTips Total Submissions: 556214
LifeProTips Total Self-Text Posts: 535212


## Data Wrangling - r/Lifehacks

Here we request data and aggregate it into a single dataframe.

In [44]:
# Content we care about:
keys = ['author', 'author_fullname', 'created_utc', 'selftext', 'title', 'subreddit', 'is_video', 'num_comments', 'score', 'upvote_ratio']

# instantiate new dict to capture api data
lh_data = defaultdict(list)

In [45]:
# making api call
lh_call = api_call('lifehacks', 100, '1648771200')

Note here we use the UTC `1648771200` which is `Friday, April 1, 2022 12:00:00 AM`. We use this date for consistency of the data we obtain.

In [46]:
len(lh_call) # we could only request data 100 submissions at a time

100

In [47]:
# wrangling api call into a dictionary that will be used on a dataframe
data = data_wrangling(lh_data, keys, lh_call)

In [48]:
# checking if any data was not capture in the api call
data['error_log']

[]

In [49]:
# api data dictionary to dataframe
df_lh = pd.DataFrame(data['data'])
df_lh.tail()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
95,Agent_Exile,t2_jsijnho7,1648637199,,true lifehack,lifehacks,False,0,1,1.0
96,satrianovian20,t2_l18xyp0p,1648636580,,useful education that will bring you closer to...,lifehacks,False,0,1,1.0
97,Scvoopy,t2_dj6kvqr3,1648631857,,Ottocast Coupon Code | 30% OFF Discount Code 2022,lifehacks,False,0,1,1.0
98,FederalBlacksmith663,t2_imeh5kjl,1648629755,[removed],Should I send a gift to my ex?,lifehacks,False,0,1,1.0
99,satrianovian20,t2_l18xyp0p,1648624604,,useful information that will bring you closer ...,lifehacks,False,0,1,1.0


A small trick here. We will use the submission time of the last post we collected, and wrangle more data that predates that submission. We will then append that data to the dataframe, until we have all the data we need. The process for that is below.

In [50]:
# last collected submission
last_utc = df_lh.loc[len(df_lh) - 1, 'created_utc']
last_utc

1648624604

Here we continue making api calls, with new data each time (predating the last data that is collected each time). We will collect at least 5000 rows of data.

In [51]:
# continue wrangling data until a certain size is met
while len(df_lh) < 5000:
    try:
        lh_call = api_call('lifehacks', 100, last_utc)
    except:
        print("Data wrangling failed.")
        break
    
    data = data_wrangling(lh_data, keys, lh_call)
    df_lh = pd.DataFrame(data['data'])
    last_utc = df_lh.loc[len(df_lh) - 1, 'created_utc']

In [52]:
# verifying data was collected
df_lh.shape

(5022, 10)

In [53]:
df_lh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5022 entries, 0 to 5021
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   author           2998 non-null   object 
 1   author_fullname  2990 non-null   object 
 2   created_utc      2998 non-null   float64
 3   selftext         2995 non-null   object 
 4   title            2998 non-null   object 
 5   subreddit        2998 non-null   object 
 6   is_video         2998 non-null   object 
 7   num_comments     2998 non-null   float64
 8   score            2998 non-null   float64
 9   upvote_ratio     2998 non-null   float64
dtypes: float64(4), object(6)
memory usage: 392.5+ KB


In [54]:
na_only(df_lh)

author_fullname    2032
selftext           2027
author             2024
created_utc        2024
title              2024
subreddit          2024
is_video           2024
num_comments       2024
score              2024
upvote_ratio       2024
dtype: int64

Looks like we have missing data. We are only interested in posts with data, so we will drop them now.

In [55]:
# drop NAs
df_lh = df_lh.dropna()

In [56]:
len(df_lh) #nunmber of rows

2990

In [57]:
# checking if our data is unique based on submission times
len(np.unique(df_lh['created_utc']))

2988

All but a few rows of data have different times of submission, which is a strong suggestion all our data are unique submissions.

In [58]:
df_lh.head()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
0,anonymousbrowzer,t2_14k10v,1.648770e+09,,"When the smoke detector goes off from cooking,...",lifehacks,False,0.0,1.0,1.0
1,PlantBasedRedditor,t2_g4e0rfz,1.648767e+09,,Use Goo Gone on scissors and blades to reduce ...,lifehacks,False,0.0,1.0,1.0
2,CryptographerFar5073,t2_ldjcr311,1.648764e+09,,Bingo Bash,lifehacks,False,0.0,1.0,1.0
3,Giant_weiner_not_dog,t2_konlr4kt,1.648763e+09,,How to troll someone,lifehacks,False,0.0,1.0,1.0
4,Giant_weiner_not_dog,t2_konlr4kt,1.648762e+09,,what a nice way to have your meal( credit to u...,lifehacks,False,0.0,1.0,1.0


In [59]:
df_lh.tail()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
2993,AnyBeing6604,t2_iag9y01b,1.642053e+09,[removed],懲罰一個男人不忠的最好方式：不是報復，也不是讓他付出代價,lifehacks,False,0.0,1.0,1.0
2994,polkmjh,t2_xrd6cwt,1.642052e+09,,buy f-phenibut powder,lifehacks,False,0.0,1.0,1.0
2995,AnyBeing6604,t2_iag9y01b,1.642052e+09,[removed],懲罰一個男人不忠的最好方式：不是報復，也不是讓他付出代價,lifehacks,False,0.0,1.0,1.0
2996,polkmjh,t2_xrd6cwt,1.642049e+09,,Dike Ajiri,lifehacks,False,0.0,1.0,1.0
2997,polkmjh,t2_xrd6cwt,1.642049e+09,,Dike Ajiri,lifehacks,False,0.0,1.0,1.0


## Data Wrangling - r/LifeProTips

We use the exact methodology to wrangle data for `r/LifeProTips` as we did with `r/lifehacks`.

In [60]:
# instantiate new dict to capture api data
lpt_data = defaultdict(list)

In [61]:
# making api call
lpt_call = api_call('LifeProTips', 100, '1648771200')

In [62]:
len(lpt_call) # we could only request data 100 submissions at a time

100

In [63]:
# wrangling api call into a dictionary that will be used on a dataframe
data = data_wrangling(lpt_data, keys, lpt_call)

Note we are using the same keys as `r/lifehacks`.

In [64]:
# checking if any data was not capture in the api call
data['error_log']

[]

In [65]:
# api data dictionary to dataframe
df_lpt = pd.DataFrame(data['data'])
df_lpt.tail()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
95,PrinceLelouch,t2_gdxku,1648735730,[removed],Save space with your Jigsaw Puzzles by putting...,LifeProTips,False,1,1,1.0
96,bmbustamante,t2_kb04e,1648735471,[removed],LPT Request: How to learn how to re-string my ...,LifeProTips,False,1,1,1.0
97,AGirlNamedPanini,t2_shu2lou,1648735345,[removed],You can have a nice bathroom to use anywhere y...,LifeProTips,False,1,1,1.0
98,cosmicucumber,t2_942tzpp,1648734733,I finally got around to painting my nails blac...,LPT: Want more compliments as a guy? Paint you...,LifeProTips,False,1,1,1.0
99,MarkGibneyNSC,t2_cspu1pkh,1648733196,,LPT: If you’ve naturally clammy hands and you’...,LifeProTips,False,1,1,1.0


In [66]:
# last collected submission
last_utc = df_lpt.loc[len(df_lpt) - 1, 'created_utc']
last_utc

1648733196

Here we continue making api calls, with new data each time (predating the last data that is collected each time). We will collect at least 1000 rows of data.

In [67]:
# continue wrangling data until a certain size is met
while len(df_lpt) < 5000:
    try:
        lpt_call = api_call('Lifeprotips', 100, last_utc)
    except:
        print("Data wrangling failed.")
        break
    
    data = data_wrangling(lpt_data, keys, lpt_call)
    df_lpt = pd.DataFrame(data['data'])
    last_utc = df_lpt.loc[len(df_lpt) - 1, 'created_utc']

In [68]:
# verifying data was collected
df_lpt.shape

(5003, 10)

In [69]:
df_lpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   author           2496 non-null   object 
 1   author_fullname  2477 non-null   object 
 2   created_utc      2496 non-null   float64
 3   selftext         2483 non-null   object 
 4   title            2496 non-null   object 
 5   subreddit        2496 non-null   object 
 6   is_video         2496 non-null   object 
 7   num_comments     2496 non-null   float64
 8   score            2496 non-null   float64
 9   upvote_ratio     2496 non-null   float64
dtypes: float64(4), object(6)
memory usage: 391.0+ KB


In [70]:
na_only(df_lpt)

author_fullname    2526
selftext           2520
author             2507
created_utc        2507
title              2507
subreddit          2507
is_video           2507
num_comments       2507
score              2507
upvote_ratio       2507
dtype: int64

Looks like we have missing data. We are only interested in posts with data, so we will drop them now.

In [71]:
# drop NAs
df_lpt = df_lpt.dropna()

In [72]:
len(df_lpt) #nunmber of rows

2477

In [73]:
# checking if our data is unique based on submission times
len(np.unique(df_lpt['created_utc']))

2477

All but a few rows of data have different times of submission, which is a strong suggestion all our data are unique submissions.

In [74]:
df_lpt.head()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
0,No-Software5654,t2_d3qolst6,1.648771e+09,Be very careful who you are surrounding yourse...,"LPT: In a world like this, don't trust anybody.",LifeProTips,False,1.0,1.0,1.0
1,A-RareEntity,t2_4puu3g6g,1.648770e+09,[removed],LPT: When you have a long drive ahead of you t...,LifeProTips,False,1.0,1.0,1.0
2,thegreatparanoia,t2_1j3hi83u,1.648769e+09,[removed],"LPT: Take 2 seconds to ask ""Can you hear me"" b...",LifeProTips,False,1.0,1.0,1.0
3,PreppingKangaroo,t2_h4nwbg1s,1.648769e+09,"Keep in mind, these prices are based on where ...",LPT: Always take advantage of sales on non-per...,LifeProTips,False,1.0,1.0,1.0
4,photomancottrell,t2_6e6siq1u,1.648768e+09,Focus your work on the areas of your house tha...,LPT: When short on time and your house needs t...,LifeProTips,False,1.0,1.0,1.0


In [75]:
df_lpt.tail()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
2491,dochoiday,t2_79f5e,1.647382e+09,[removed],"Unless it is covered under warranty, DO NOT br...",LifeProTips,False,1.0,1.0,1.0
2492,devilenka,t2_8kn983ie,1.647382e+09,,LPT: Be careful about how much you share at wo...,LifeProTips,False,1.0,1.0,1.0
2493,gladeyes,t2_erudf,1.647381e+09,,LPT. Sometimes it’s easier to figure out what ...,LifeProTips,False,1.0,1.0,1.0
2494,Optimistprime777,t2_3z31bmer,1.647381e+09,,LPT: Clean up your YouTube subscriptions from ...,LifeProTips,False,1.0,1.0,1.0
2495,ContributionNarrow88,t2_6982d42g,1.647380e+09,,LPT: Never attack someone else's character if ...,LifeProTips,False,1.0,1.0,1.0


-----

### Merging Data

In [76]:
# "stacking" both dataframes by row using concatenate method
df_all = pd.concat([df_lh, df_lpt], axis=0)

In [77]:
# must reset repeated indices after concatenation
df_all.reset_index(drop=True, inplace=True)

In [78]:
df_all.shape

(5467, 10)

In [79]:
df_all.head()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
0,anonymousbrowzer,t2_14k10v,1.648770e+09,,"When the smoke detector goes off from cooking,...",lifehacks,False,0.0,1.0,1.0
1,PlantBasedRedditor,t2_g4e0rfz,1.648767e+09,,Use Goo Gone on scissors and blades to reduce ...,lifehacks,False,0.0,1.0,1.0
2,CryptographerFar5073,t2_ldjcr311,1.648764e+09,,Bingo Bash,lifehacks,False,0.0,1.0,1.0
3,Giant_weiner_not_dog,t2_konlr4kt,1.648763e+09,,How to troll someone,lifehacks,False,0.0,1.0,1.0
4,Giant_weiner_not_dog,t2_konlr4kt,1.648762e+09,,what a nice way to have your meal( credit to u...,lifehacks,False,0.0,1.0,1.0


In [80]:
df_all.tail()

,author,author_fullname,created_utc,selftext,title,subreddit,is_video,num_comments,score,upvote_ratio
5462,dochoiday,t2_79f5e,1.647382e+09,[removed],"Unless it is covered under warranty, DO NOT br...",LifeProTips,False,1.0,1.0,1.0
5463,devilenka,t2_8kn983ie,1.647382e+09,,LPT: Be careful about how much you share at wo...,LifeProTips,False,1.0,1.0,1.0
5464,gladeyes,t2_erudf,1.647381e+09,,LPT. Sometimes it’s easier to figure out what ...,LifeProTips,False,1.0,1.0,1.0
5465,Optimistprime777,t2_3z31bmer,1.647381e+09,,LPT: Clean up your YouTube subscriptions from ...,LifeProTips,False,1.0,1.0,1.0
5466,ContributionNarrow88,t2_6982d42g,1.647380e+09,,LPT: Never attack someone else's character if ...,LifeProTips,False,1.0,1.0,1.0


In [81]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5467 entries, 0 to 5466
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   author           5467 non-null   object 
 1   author_fullname  5467 non-null   object 
 2   created_utc      5467 non-null   float64
 3   selftext         5467 non-null   object 
 4   title            5467 non-null   object 
 5   subreddit        5467 non-null   object 
 6   is_video         5467 non-null   object 
 7   num_comments     5467 non-null   float64
 8   score            5467 non-null   float64
 9   upvote_ratio     5467 non-null   float64
dtypes: float64(4), object(6)
memory usage: 427.2+ KB


In [82]:
na_only(df_all)

0

Everything looks good! We can now export.

## Data Exporting

Uncomment cell below to export CSV.

**NOTE**: Reddit is a living forum, and the state of the data is likely to change day-to-day. It is likely that you may wrangle data that is different from the one presented here, which may have drastic effects on the models in the next notebooks. Data that was wrangled when this notebook was created is stored in as `../datasets/submissions_data.csv`. If you wish to replicate the results exactly as we did here, please use that dataset.

In [83]:
#df_all.to_csv('../datasets/submissions_data.csv', index=False)